To do:


1.   Test more hyperparameters to get metrics in 0.9 - HuggingFace Trainer has native support for hyperparameter search using either Optuna, Ray Tune, or Weights & Biases.
2.   Data augmentation - use another LLM to do NER categorization of text. and then add that to the training model.



Another relevant application of large language models (LLMs) in linguistic tasks is Named Entity Recognition (NER). Recent work by Beersmans et al. (2024) demonstrates this by combining transformer-based models with domain-specific knowledge to identify individuals in Ancient Greek texts. Their study, “Gotta catch ’em all!: Retrieving people in Ancient Greek texts combining transformer models and domain knowledge,” was presented at the 1st Workshop on Machine Learning for Ancient Languages (ML4AL 2024) and provides a strong example of how modern NLP techniques can be adapted for historical languages.

The authors built a model for Ancient greek NER task with F1 score of 0.826, which is State-of-Art as of current. Since Ancient Greek is a low-resource, highly inflected ancient language with limited annotated corpora (e.g., the dataset used here aggregates ~5,579 test tokens from projects like First1KGreek and NERAncientGreekML4AL). Unlike modern high-resource languages (e.g., English, where CoNLL-2003 NER F1 scores exceed 0.93), ancient languages suffer from data scarcity, orthographic variations (e.g., diacritics, dialects), and domain noise (e.g., fragmentary inscriptions or papyri). SOTA in this niche is typically in the 0.80–0.89 range for transformer-based models on similar tasks.

We attempted to do hyperparameters tuning for a better performance - there are 2 hyperparameters not tested in the original paper - Warmup ratio and batch size. Both of these are sensitive to transformer tuning.
Large batch size reduces noise, resulting in better token representation. This is particulary useful for complex morphologically rich languages like Ancient Greek. However, smaller batches tend to act like regularization allowing for beter generalization. Thus tuning of the batch size is to find the balance between overfitting and token representation.

[describe warm up]

Data augmentation is another potential approach to improving the model’s F1 score, but it is not practical for this project. Given our limited resources, we would need multiple models to first annotate the English translations of the Ancient Greek sentences and then align those annotations back to the corresponding Koine Greek tokens to infer NER labels. Even after this automated pipeline, human verification would still be required to ensure label accuracy. Producing a dataset of roughly 100,000 Koine Greek tokens under these constraints would be extremely time-consuming and effectively not feasible within the scope of this project.

In [4]:
#!pip install --upgrade transformers
!pip install -q transformers datasets seqeval torch tqdm evaluate


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:


# Clone the repo
#!git clone https://github.com/NER-AncientLanguages/NERAncientGreekML4AL.git
#%cd NERAncientGreekML4AL

# Verify data exists
#!ls final_dataset/normal/*.conll

Cloning into 'NERAncientGreekML4AL'...
remote: Enumerating objects: 251, done.
remote: Total 251 (delta 0), reused 0 (delta 0), pack-reused 251 (from 3)
Receiving objects: 100% (251/251), 106.47 MiB | 16.09 MiB/s, done.
Resolving deltas: 100% (98/98), done.
Updating files: 100% (199/199), done.
Error downloading object: Data/homogenisation/full_dataset_FINAL.csv (82d984c): Smudge error: Error downloading Data/homogenisation/full_dataset_FINAL.csv (82d984c506fbdcea63db80edc6d34c42f5128b2de3a34df706c6ecae87f02254): batch response: This repository exceeded its LFS budget. The account responsible for the budget should increase it to restore access.

Errors logged to /content/NERAncientGreekML4AL/.git/lfs/logs/20251118T021904.003062136.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: Data/homogenisation/full_dataset_FINAL.csv: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore 

In [12]:
import os, warnings, unicodedata, numpy as np
from pathlib import Path
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification,
    TrainingArguments, Trainer, DataCollatorForTokenClassification
)

import evaluate
from seqeval.metrics import classification_report

def read_conll(p: Path):
    """
    Parse CoNLL with format:
        [line_id]  token  [POS]  NER
    Example:
        110089790	βίβλος	O
    Returns: {"tokens": [...], "ner_tags": [...]}
    """
    sents, labs = [], []
    with p.open(encoding="utf-8") as f:
        sent, lab = [], []
        for i, raw in enumerate(f, 1):
            line = raw.strip()
            if not line or line.startswith("#"):
                if sent:
                    sents.append(sent)
                    labs.append(lab)
                    sent, lab = [], []
                continue

            # Split on whitespace (handles tabs and spaces)
            parts = line.split()
            if len(parts) < 2:
                print(f"Warning: Line {i} in {p.name} has <2 columns → SKIPPED")
                print(f"    → {line!r}")
                continue

            if len(parts) == 2:
                token = parts[0]
                ner   = parts[1]
            else:
                token = parts[1]   # skip ID
                ner   = parts[-1]  # last column is NER

            sent.append(unicodedata.normalize("NFC", token))
            lab.append(ner)

        if sent:
            sents.append(sent)
            labs.append(lab)

    print(f"Loaded {len(sents)} sentences from {p.name}")
    return {"tokens": sents, "ner_tags": labs}

# load data
train_path = Path("/content/drive/My Drive/Deep Learning Group Project/train.conll")
val_path   = Path("/content/drive/My Drive/Deep Learning Group Project/val.conll")
test_path  = Path("/content/drive/My Drive/Deep Learning Group Project/test.conll")

raw = {
    "train": read_conll(train_path),
    "validation": read_conll(val_path),
    "test": read_conll(test_path),
}
data = DatasetDict({k: Dataset.from_dict(v) for k, v in raw.items()})

#Model name -------------------------------------------------------------
model_name = "Marijke/AG_BERT_hypopt_NER"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
#------------------------------------------------------------------------

all_labels = sorted({l for s in data["train"]["ner_tags"] for l in s})
label2id   = {l: i for i, l in enumerate(all_labels)}
id2label   = {i: l for l, i in label2id.items()}

#tokenise + align labels
def tokenise_align(example):
    tok = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    aligned = []
    for i, labs in enumerate(example["ner_tags"]):
        word_ids = tok.word_ids(batch_index=i)
        prev = None
        ids  = []
        for wid in word_ids:
            if wid is None:
                ids.append(-100)
            elif wid != prev:
                ids.append(label2id[labs[wid]])
            else:
                ids.append(-100)               # sub-word → ignore
            prev = wid
        aligned.append(ids)
    tok["labels"] = aligned
    return tok

tokenised = data.map(tokenise_align, batched=True,
                     remove_columns=data["train"].column_names)


model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(all_labels),
    id2label=id2label,
    label2id=label2id,
)

collator = DataCollatorForTokenClassification(tokenizer)


def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=2)

    true_labels = []
    pred_labels = []

    for prediction, label in zip(preds, labels):
        true_seq = [id2label[l] for l in label if l != -100]
        pred_seq = [id2label[pred] for pred, l in zip(prediction, label) if l != -100]
        if true_seq:  # Only add if not empty
            true_labels.append(true_seq)
            pred_labels.append(pred_seq)

    if not true_labels:
        return {"precision": 0.0, "recall": 0.0, "f1": 0.0}


    metric = evaluate.load("seqeval")
    results = metric.compute(predictions=pred_labels, references=true_labels)

    return {
      "precision": results["overall_precision"],
      "recall": results['overall_recall'],
      "f1": results["overall_f1"]
    }



Loaded 30686 sentences from train.conll
Loaded 4434 sentences from val.conll
Loaded 4701 sentences from test.conll


Map:   0%|          | 0/30686 [00:00<?, ? examples/s]

Map:   0%|          | 0/4434 [00:00<?, ? examples/s]

Map:   0%|          | 0/4701 [00:00<?, ? examples/s]

Hyperparameter tuning was performed using Hyperopt. Although Hyperopt is less commonly used today, we chose it to maintain consistency with the methodology described in the referenced paper.

In [22]:
import os
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification,
)
import evaluate
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, STATUS_FAIL
from hyperopt.early_stop import no_progress_loss

# -------------------------------
# HYPEROPT SEARCH SPACE
# Include 2 new parameters that were not tried in the paper - batch size and warmup ratio
# -------------------------------
FIXED_LR = 6.040686648207059e-05
FIXED_WD = 0.01
FIXED_EPOCH = 3

space = {
    "batch_size":    hp.choice("batch_size", [8, 16, 32]),            # 3 options
    "warmup_ratio":  hp.choice("warmup_ratio", [0.0, 0.06, 0.1, 0.2]), # 4 options
    "seed": 123 #for reproducibility
}

# -------------------------------
# OBJECTIVE FUNCTION used by Hyperopt to test parameters
# -------------------------------
def objective(params):
  # we are keeping these 3 hyperparameters from the paper itself as they have found the optimal values for the Learning Rate, Weight Decay
  # and number of training epoch

    try:

        # Corrected: Directly use the values from params, as hp.choice returns the value itself, not an index
        batch_size = params["batch_size"]
        warmup_ratio = params["warmup_ratio"]

        model_for_trial = AutoModelForTokenClassification.from_pretrained(
            model_name,
            num_labels=len(all_labels),
            id2label=id2label,
            label2id=label2id,
        )


        total_steps = int(len(tokenised["train"]) / batch_size * FIXED_EPOCH)
        warmup_steps = int(total_steps * warmup_ratio)

        training_args = TrainingArguments(
            output_dir=f"./hyperopt_trial_{int(FIXED_EPOCH)}_{batch_size}_{FIXED_LR:.2e}",
            num_train_epochs=FIXED_EPOCH,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size * 2,
            learning_rate=FIXED_LR,
            weight_decay=FIXED_WD,
            warmup_steps=warmup_steps,
            lr_scheduler_type="linear",
            eval_strategy="epoch",
            save_strategy="epoch",
            logging_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            greater_is_better=True,
            report_to="none",
            seed=params["seed"],
            dataloader_num_workers=4,
            disable_tqdm=False,
        )

        trainer = Trainer(
            model=model_for_trial,
            args=training_args,
            train_dataset=tokenised["train"],
            eval_dataset=tokenised["validation"],
            tokenizer=tokenizer,
            data_collator=collator,
            compute_metrics=compute_metrics,
        )

        trainer.train()
        metrics = trainer.evaluate()

        return {
            "loss": -metrics["eval_f1"],
            "status": STATUS_OK,
            "eval_f1": metrics["eval_f1"],
            "params": params,
        }

    except Exception as e:
        print(f"Trial failed: {e}")
        return {"loss": 10.0, "status": STATUS_FAIL}

# -------------------------------
# RUN HYPEROPT
# -------------------------------
trials = Trials()

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=12,
    trials=trials,
    rstate=np.random.default_rng(42),
    show_progressbar=True,
)

# -------------------------------
# PRINT BEST RESULT
# -------------------------------
best_trial = trials.best_trial
print("\n" + "="*60)
print("BEST HYPERPARAMETERS FOUND")
print("="*60)
print(f"Best eval micro F1 : {best_trial['result']['eval_f1']:.4f}")
print(f"Batch size         : {int(best_trial['result']['params']['batch_size'])}")
print(f"Warmup ratio       : {best_trial['result']['params']['warmup_ratio']}")
print("="*60)

# Optional: retrain on full train+val with best params and evaluate on test set

  0%|          | 0/12 [00:00<?, ?trial/s, best loss=?]

/tmp/ipython-input-1339185124.py:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.035000,0.095455,0.826412,0.841854,0.834061
2,0.020800,0.104416,0.825974,0.852765,0.839156
3,0.011400,0.118280,0.828293,0.851570,0.839770


  8%|▊         | 1/12 [11:41<2:08:33, 701.21s/trial, best loss: -0.8397700471698114]

/tmp/ipython-input-1339185124.py:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.051500,0.103613,0.818583,0.804634,0.811548
2,0.034300,0.109292,0.823027,0.835575,0.829254
3,0.016700,0.124583,0.833012,0.840359,0.836669


 92%|█████████▏| 11/12 [2:15:30<12:37, 757.46s/trial, best loss: -0.8424385215726697]

/tmp/ipython-input-1339185124.py:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.032200,0.105150,0.820698,0.833333,0.826967
2,0.025100,0.101871,0.819980,0.842900,0.831282
3,0.012900,0.114957,0.828934,0.849626,0.839153


100%|██████████| 12/12 [2:27:14<00:00, 736.17s/trial, best loss: -0.8424385215726697]

BEST HYPERPARAMETERS FOUND
Best eval micro F1 : 0.8424
Batch size         : 32
Warmup ratio       : 0.1


In [24]:
# Create a model with the best hyper parameters found.
# ------------------------------------------------------------
# Hyper-parameters
# ------------------------------------------------------------
LEARNING_RATE = 6.040686648207059e-05 #From paper
EPOCHS        = 3                     #From paper
WEIGHT_DECAY  = 0.01                   #From paper
BATCH_SIZE    = 32                     #Best parameter from above
WARMUP_RATIO  = 0.1                    #Best parameter from above
SEED          = 123
OUTPUT_DIR    = "/content/drive/My Drive/Deep Learning Group Project/tuned_ner_model"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=SEED,
    logging_steps=10,
    save_total_limit=2,
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenised["train"],
    eval_dataset=tokenised["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

#Train the model
print("\nSTARTING TRAINING ...\n")
trainer.train()

#Save the model
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"\nModel saved to {OUTPUT_DIR}")



/tmp/ipython-input-2008623300.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



STARTING TRAINING ...



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.030600,0.097973,0.827334,0.827952,0.827643
2,0.022800,0.104885,0.826486,0.847982,0.837096
3,0.017000,0.113758,0.832677,0.853214,0.842820



Model saved to /content/drive/My Drive/Deep Learning Group Project/tuned_ner_model


In [25]:
#Quick test
from transformers import pipeline
import unicodedata

ner = pipeline("ner", model=OUTPUT_DIR, tokenizer=OUTPUT_DIR,
               aggregation_strategy="simple")

txt = unicodedata.normalize("NFC", """
  ᾿Ανέστη δὲ βασιλεὺς ἕτερος ἐπ᾿ Αἴγυπτον, ὃς οὐκ ᾔδει τὸν ᾿Ιωσήφ.
  εἶπε δὲ τῷ ἔθνει αὐτοῦ· ἰδοὺ τὸ γένος τῶν υἱῶν ᾿Ισραὴλ μέγα πλῆθος καὶ ἰσχύει ὑπὲρ ἡμᾶς·
  δεῦτε οὖν κατασοφισώμεθα αὐτούς, μή ποτε πληθυνθῇ, καὶ ἡνίκα ἂν συμβῇ ἡμῖν πόλεμος,
  προστεθήσονται καὶ οὗτοι πρὸς τοὺς ὑπεναντίους καὶ ἐκπολεμήσαντες ἡμᾶς ἐξελεύσονται ἐκ τῆς γῆς.
  καὶ ἐπέστησεν αὐτοῖς ἐπιστάτας τῶν ἔργων, ἵνα κακώσωσιν αὐτοὺς ἐν τοῖς ἔργοις· καὶ Ισραήλᾠκοδόμησαν πόλεις ὀχυρὰς τῷ Φαραώ, τήν τε Πειθὼ καὶ Ῥαμεσσῆ καὶ ῎Ων, ἥ ἐστιν ῾Ηλιούπολις.
  καθότι δὲ αὐτοὺς ἐταπείνουν, τοσούτῳ πλείους ἐγίγνοντο, καὶ ἴσχυον σφόδρα σφόδρα· καὶ ἐβδελύσσοντο οἱ Αἰγύπτιοι ἀπὸ τῶν υἱῶν ᾿.
  καὶ κατεδυνάστευον οἱ Αἰγύπτιοι τοὺς υἱοὺς ᾿Ισραὴλ βίᾳ καὶ κατωδύνων αὐτῶν τὴν ζωὴν ἐν τοῖς ἔργοις τοῖς σκληροῖς, τῷ πηλῷ καὶ τῇ πλινθείᾳ καὶ πᾶσι τοῖς ἔργοις τοῖς ἐν τοῖς πεδίοις, κατὰ πάντα τὰ ἔργα, ὧν κατεδουλοῦντο αὐτοὺς μετὰ βίας.
""")

merged_results = []

for r in ner(txt):
    if r['word'].startswith("##"):
        merged_results[-1]['word'] += r['word'][2:]  # remove ## and join the subwords together instead of splitting it
        merged_results[-1]['score'] = max(merged_results[-1]['score'], r['score'])
    else:
        merged_results.append(r)

for r in merged_results:
    print(f"{r['word']:<20} → {r['entity_group']:<6} ({r['score']:.3f})")


Device set to use cuda:0


αιγυπτον             → LOC    (0.991)
φαραω                → PERS   (0.998)
ραμεσση              → PERS   (0.837)
αιγυπτιοι            → GRP    (0.999)
αιγυπτιοι            → GRP    (0.999)


In [27]:
FINAL_MODEL_DIR = OUTPUT_DIR
tokenizer_test = AutoTokenizer.from_pretrained(FINAL_MODEL_DIR)
model_test = AutoModelForTokenClassification.from_pretrained(FINAL_MODEL_DIR)

trainer_test = Trainer(
    model=model_test,
    args=TrainingArguments(
        output_dir="./temp_eval",
        per_device_eval_batch_size=32,
    ),
    eval_dataset=tokenised['test'], #used the Test dataset that was previously processed in same manner as the Train and Val
    tokenizer=tokenizer_test,
    data_collator=DataCollatorForTokenClassification(tokenizer_test),
    compute_metrics=compute_metrics,
)

print("Running official test set evaluation...")
results = trainer.evaluate()

print("\n" + "═" * 60)
print("FINAL OFFICIAL TEST RESULTS (same as paper)")
print("═" * 60)
print(f"Precision : {results['eval_precision']:.4f}")
print(f"Recall    : {results['eval_recall']:.4f}")
print(f"Micro F1  : {results['eval_f1']:.4f}")
print("═" * 60)

if results['eval_f1'] > 0.826:
    print("We did better than the paper's 0.826!")
else:
    print("Close to or matches the original paper result.")

/tmp/ipython-input-706216770.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_test = Trainer(


Running official test set evaluation...



════════════════════════════════════════════════════════════
FINAL OFFICIAL TEST RESULTS (same as paper)
════════════════════════════════════════════════════════════
Precision : 0.8327
Recall    : 0.8532
Micro F1  : 0.8428
════════════════════════════════════════════════════════════
We did better than the paper's 0.826!


In [ ]:
# ------------------------------------------------------------
# Hyper-parameters
# ------------------------------------------------------------
LEARNING_RATE = 3e-5
BATCH_SIZE    = 32
EPOCHS        = 5
WEIGHT_DECAY  = 0.01
WARMUP_RATIO  = 1.0
SEED          = 123
OUTPUT_DIR    = f"./tuned_ner_model_lr{LEARNING_RATE}_bs{BATCH_SIZE}_ep{EPOCHS}"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=SEED,
    logging_steps=10,
    save_total_limit=2,
    report_to=[],
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenised["train"],
    eval_dataset=tokenised["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Reference:
Beersmans, M., Keersmaekers, A., de Graaf, E., Van de Cruys, T., Depauw, M., & Fantoli, M. (2024). “Gotta catch ’em all!”: Retrieving people in Ancient Greek texts combining transformer models and domain knowledge. In J. Pavlopoulos et al. (Eds.), Proceedings of the 1st Workshop on Machine Learning for Ancient Languages (ML4AL 2024) (pp. 152–164). Association for Computational Linguistics. https://doi.org/10.18653/v1/2024.ml4al-1.16